**Import**

In [ ]:
from sklearnex import patch_sklearn, unpatch_sklearn
patch_sklearn()

import argparse
import random
import math
import csv
import pickle
import cv2
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from skimage import io, color
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from scipy.signal import convolve2d

from skimage.feature import local_binary_pattern


# from skimage.color import rgb2gray
# import imageio
import skimage.feature

# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import VotingClassifier
# from sklearn.neighbors import KNeighborsClassifier

import pywt

import mahotas
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


**Def Features**

In [ ]:
class FeatureExtractor:
    def __init__(self):
        pass

    def predict(self, img):
        pass

    def featureExtract(self, imgs):
        feature = []
        for i in tqdm(range(0, imgs.shape[0])):
            dense_feat = self.predict(imgs[i])
            feature.append(dense_feat)

        scaler = StandardScaler()
        scaler.fit(feature)
        normalized_feature = scaler.transform(feature)

        return np.array(normalized_feature)
    

class Color(FeatureExtractor):
    def __init__(self):
        super().__init__()

    def predict(self, img):
        # img = img.reshape(img.shape[0] * img.shape[1], -1)
        # return np.mean(img, axis=0)
    
        pixels = np.array(img)

        # Quantize colors to, say, 64 colors
        # Reduce each color channel into 4 bins (4*4*4 = 64 colors)
        pixels_quantized = (pixels // 64) * 64

        # Create histogram
        hist, _ = np.histogramdd(pixels_quantized.reshape(-1, 3), bins=(4, 4, 4), range=((0, 256), (0, 256), (0, 256)))

        # Flatten histogram to use as a feature vector
        feature_vector = hist.flatten()

        # Normalize the histogram
        feature_vector /= feature_vector.sum()

        return feature_vector

class Raw(FeatureExtractor):
    def __init__(self):
        super().__init__()

    def predict(self, img):
        return img.flatten()
    

class SmoothRaw(FeatureExtractor):

    def __init__(self, size=16):
        super().__init__()
        self.size = size

    def predict(self, img):
        b, g, r = cv2.split(img)
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        resized_grey_image = cv2.resize(gray_image, (self.size, self.size), interpolation=cv2.INTER_AREA)
        resized_b = cv2.resize(b, (self.size, self.size), interpolation=cv2.INTER_AREA)
        resized_g = cv2.resize(g, (self.size, self.size), interpolation=cv2.INTER_AREA)
        resized_r = cv2.resize(r, (self.size, self.size), interpolation=cv2.INTER_AREA)

        return np.concatenate((resized_grey_image, resized_b, resized_g, resized_r)).flatten()


class cv2HOG(FeatureExtractor):
    def __init__(self, win_size=(32, 32), block_size=(16, 16), block_stride=(8, 8), cell_size=(8, 8), num_bins=20):
        super().__init__()
        self.win_size = win_size
        self.block_size = block_size
        self.block_stride = block_stride
        self.cell_size = cell_size
        self.num_bins = num_bins
        self.hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, num_bins)

    def predict(self, img):
        b_img, g_img, r_img = cv2.split(img)
        grey_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        features_b = self.hog.compute(b_img)
        features_g = self.hog.compute(g_img)
        features_r = self.hog.compute(r_img)
        features_grey = self.hog.compute(grey_image)

        return np.concatenate((features_grey, features_b, features_g, features_r), axis=0)


class sklearnHOG(FeatureExtractor):
    def __init__(self, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), channel_axis=2):
        super().__init__()
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.channel_axis = channel_axis

    def predict(self, img):
        # b_img, g_img, r_img = cv2.split(img)
        # grey_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        feature = hog(img, orientations=self.orientations, pixels_per_cell=self.pixels_per_cell, cells_per_block=self.cells_per_block, channel_axis=self.channel_axis)
        # features_b = self.hog.compute(b_img)
        # features_g = self.hog.compute(g_img)
        # features_r = self.hog.compute(r_img)
        # features_grey = self.hog.compute(grey_image)

        return feature
    

class SIFT(FeatureExtractor):

    def __init__(self):
        super().__init__()


    def predict(self, img):
        gray_im = color.rgb2gray(img)
        sift = cv2.SIFT_create()
        _, descriptors = sift.detectAndCompute((gray_im * 255).astype("uint8"), None)

        if descriptors is None:
            return np.zeros((128,))
        

        return np.mean(descriptors, axis=0)



class HuMoments(FeatureExtractor):
    def __init__(self):
        super().__init__()

    def predict(self, img):
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        moments = cv2.moments(img_gray)
        humoments = cv2.HuMoments(moments)
        humoments = -(np.log(np.abs(humoments))) / np.log(10)
        return humoments.flatten()
    

class EOH(FeatureExtractor):
    def __init__(self, num_blocks=8):
        super().__init__()
        self.num_blocks = num_blocks

    def get_sobel_filters(self):
        # Define Sobel filters for horizontal, vertical, 45-degree, and 135-degree edges
        return [
            np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float32),  # Horizontal
            np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], dtype=np.float32),  # Vertical
            np.array([[2, 1, 0], [1, 0, -1], [0, -1, -2]], dtype=np.float32),  # 45-degree
            np.array([[0, 1, 2], [-1, 0, 1], [-2, -1, 0]], dtype=np.float32)  # 135-degree
        ]

    def apply_filters(self, image, filters):
        # Apply each filter to the image
        edge_maps = [convolve2d(image, filter, mode='same', boundary='wrap') for filter in filters]
        return edge_maps

    def calculate_histograms(self, image, edge_maps):
        # Assume the image is divided into num_blocks x num_blocks
        block_size = image.shape[0] // self.num_blocks
        histograms = np.zeros((self.num_blocks, self.num_blocks, len(edge_maps)), dtype=np.float32)

        for i in range(self.num_blocks):
            for j in range(self.num_blocks):
                block = [em[i * block_size:(i + 1) * block_size, j * block_size:(j + 1) * block_size] for em in
                         edge_maps]
                histograms[i, j] = [np.sum(np.abs(b)) for b in block]

        return histograms

    def predict(self, img):
        # Convert the image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Get the Sobel filters
        filters = self.get_sobel_filters()

        # Apply the filters to the image
        edge_maps = self.apply_filters(gray, filters)

        # Calculate the histograms
        histograms = self.calculate_histograms(gray, edge_maps)

        # normalize the histograms
        histograms /= histograms.sum(axis=(0, 1), keepdims=True)

        # Flatten the histograms
        return histograms.flatten()
    

class LBP(FeatureExtractor):
    def __init__(self, num_points=8, radius=1):
        super().__init__()
        self.num_points = num_points
        self.radius = radius

    def predict(self, img, P=8, R=1):
        # Convert the image to grayscale as LBP works on grayscale images
        # Convert the image to grayscale
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Compute the LBP features
        lbp_image = local_binary_pattern(gray_image, 8, 1, method='uniform')

        # Compute the histogram of LBP features
        histogram, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, 60), range=(0, 59))

        # Normalize the histogram
        histogram = histogram.astype(np.float32)
        histogram /= (histogram.sum() + 1e-5)

        return histogram
    

class cooccurrence(FeatureExtractor):
    def __init__(self):
        super().__init__()


    def predict(self, img):
        # Convert the image to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        co_matrix = skimage.feature.graycomatrix(img, [5], [0], levels=256, symmetric=True, normed=True)

        # Calculate texture features from the co-occurrence matrix
        contrast = skimage.feature.graycoprops(co_matrix, 'contrast')
        correlation = skimage.feature.graycoprops(co_matrix, 'correlation')
        energy = skimage.feature.graycoprops(co_matrix, 'energy')
        homogeneity = skimage.feature.graycoprops(co_matrix, 'homogeneity')

        return np.array([contrast[0, 0], correlation[0, 0], energy[0, 0], homogeneity[0, 0]]).flatten()
    

class Gabor(FeatureExtractor):
    def __init__(self):
        super().__init__()

    def predict(self, img):
        # Convert the image to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Define the range of orientations
        orientations = [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4]

        # Define the range of frequencies
        frequencies = [0.1, 0.4, 0.7]

        # Create a Gabor filter for each orientation and frequency
        filters = [cv2.getGaborKernel((21, 21), 4.0, theta, freq, 0.5, 0, ktype=cv2.CV_32F) for theta in orientations for freq in frequencies]

        # Apply each filter to the image
        responses = [cv2.filter2D(img, cv2.CV_32F, filter) for filter in filters]

        # Calculate the mean and standard deviation of the responses
        mean = np.array([response.mean() for response in responses])
        std = np.array([response.std() for response in responses])

        return np.concatenate((mean, std)).flatten()
    

class Tamura(FeatureExtractor):
    def __init__(self):
        super().__init__()

    def predict(self, img):
        # Convert the image to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Calculate the coarseness, contrast, and directionality
        coarseness = self.calculate_coarseness(img)
        contrast = self.calculate_contrast(img)
        directionality = self.calculate_directionality(img)

        return np.array([coarseness, contrast, directionality]).flatten()

    def calculate_coarseness(self, img):
        # Calculate the gradient of the image
        gradient = cv2.Sobel(img, cv2.CV_64F, 1, 1, ksize=5)

        # Calculate the mean of the gradient
        mean = np.abs(gradient).mean()

        # Calculate the coarseness
        coarseness = mean / 4.0

        return coarseness

    def calculate_contrast(self, img):
        # Calculate the contrast
        contrast = img.std()

        return contrast

    def calculate_directionality(self, img):
        # Calculate the gradient of the image
        gradient = cv2.Sobel(img, cv2.CV_64F, 1, 1, ksize=5)

        # Calculate the histogram of the gradient
        histogram, _ = np.histogram(gradient.ravel(), bins=256, range=(-128, 128))

        # Normalize the histogram
        histogram = histogram.astype(np.float32)
        histogram /= (histogram.sum() + 1e-5)

        # Calculate the entropy of the histogram
        entropy = -np.sum(histogram * np.log2(histogram + 1e-5))

        # Calculate the directionality
        directionality = 1 - 1 / (1 + entropy)

        return directionality


class Daubechies_Wavelets(FeatureExtractor):
    def __init__(self):
        super().__init__()

    def predict(self, img):
        # Convert the image to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Perform the Daubechies wavelet transform
        coeffs = pywt.dwt2(img, 'db1')

        # Calculate the mean and standard deviation of the coefficients
        mean = [np.mean(c) for c in coeffs]
        std = [np.std(c) for c in coeffs]

        return np.concatenate((mean, std)).flatten()
    

class ZernikeMoments(FeatureExtractor):
    def __init__(self):
        super().__init__()

    def predict(self, img):
        # Convert the image to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Calculate the Zernike moments
        moments = mahotas.features.zernike_moments(img, radius=21, degree=8)

        return moments.flatten()
    

class Haralick(FeatureExtractor):
    def __init__(self):
        super().__init__()

    def predict(self, img):
        # Convert the image to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Calculate the Haralick texture features
        features = mahotas.features.haralick(img).mean(axis=0)

        return features.flatten()
    

class GIST(FeatureExtractor):
    def __init__(self):
        super().__init__()

    def predict(self, img):
        # Convert the image to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Calculate the GIST features
        features = mahotas.features.haralick(img).mean(axis=0)

        return features.flatten()
    

class DenseSIFT(FeatureExtractor):
    def __init__(self):
        super().__init__()

    def predict(self, img):
        gray_im = color.rgb2gray(img)
        sift = cv2.SIFT_create()
        _, descriptors = sift.detectAndCompute((gray_im * 255).astype("uint8"), None)

        if descriptors is None:
            return np.zeros((128,))
        

        return np.mean(descriptors, axis=0)
     

class DCT(FeatureExtractor):
    def __init__(self):
        super().__init__()

    def predict(self, img):
        # Convert the image to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Perform the 2D DCT
        dct = cv2.dct(np.float32(img))

        return dct.flatten()


def random_rotate(img, degrees_range=(-10, 10)):
    angle = random.uniform(degrees_range[0], degrees_range[1])
    height, width = img.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    rotated_image = cv2.warpAffine(img, rotation_matrix, (width, height))
    return rotated_image


**Load Training Features**

In [10]:
image_names = []
labels = []
test_image_names = []
test_labels = []

train_csv_file = './train.csv'
test_csv_file = './test.csv'

with open(train_csv_file, 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)
    for row in csv_reader:
        image_names.append(row[0])
        labels.append(int(row[1]))


imgs = []
for i, image_name in enumerate(image_names):
    images_path = f'./train_ims/{image_name}'
    img = cv2.imread(images_path)
    imgs.append(img)
print(len(imgs))

print("begin data augmentation")
for i in range(len(imgs)):
    img = imgs[i]
    # flip
    img = cv2.flip(img, 1)
    imgs.append(img)
    labels.append(labels[i])
    # # rotate
    # img = random_rotate(img)
    # imgs.append(img)
    # labels.append(labels[i])
print(len(imgs))

print("data augmentation done")

with open(test_csv_file, 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)
    for row in csv_reader:
        test_image_names.append(row[0])
        labels.append(int(row[1]))


for i, image_name in enumerate(test_image_names):
    images_path = f'./test_ims/{image_name}'
    img = cv2.imread(images_path)
    imgs.append(img)
print(len(imgs))

print("finished loading")



for i in range(10):
    print("Number of images with label", i, ":", labels.count(i))

# data = list(zip(imgs, labels))
# random.shuffle(data)
# imgs, labels = zip(*data)

imgs = np.array(imgs)
labels = np.array(labels)


50000
begin data augmentation
100000
data augmentation done
110000
finished loading
Number of images with label 0 : 20020
Number of images with label 1 : 10024
Number of images with label 2 : 10076
Number of images with label 3 : 10014
Number of images with label 4 : 9990
Number of images with label 5 : 9986
Number of images with label 6 : 9910
Number of images with label 7 : 10000
Number of images with label 8 : 10040
Number of images with label 9 : 9940


**Save Features**

In [11]:
# skHOG_extractor = sklearnHOG()
# cvHOG_extractor = cv2HOG()
# Color_extractor = Color()
# Raw_extractor = Raw()
# smoothRaw_extractor = SmoothRaw()
# EOH_extractor = EOH()
# HuMoments_extractor = HuMoments()
# SIFT_extractor = SIFT()
# LBP_extractor = LBP()
# cooccurrence_extractor = cooccurrence()
# Gabor_extractor = Gabor()
# Tamura_extractor = Tamura()
# Daubechies_Wavelets_extractor = Daubechies_Wavelets()
# ZernikeMoments_extractor = ZernikeMoments()
# Haralick_extractor = Haralick()
# GIST_extractor = GIST()
# DenseSIFT_extractor = DenseSIFT()
# DCT_extractor = DCT()


 

# cvHOG_feature = cvHOG_extractor.featureExtract(imgs)
# Color_feature = Color_extractor.featureExtract(imgs)
# raw_feature = Raw_extractor.featureExtract(imgs)
# EOH_feature = EOH_extractor.featureExtract(imgs)
# HuMoments_feature = HuMoments_extractor.featureExtract(imgs)
# SIFT_feature = SIFT_extractor.featureExtract(imgs)
# LBP_feature = LBP_extractor.featureExtract(imgs)
# cooccurrence_feature = cooccurrence_extractor.featureExtract(imgs)
# Gabor_feature = Gabor_extractor.featureExtract(imgs)
# Tamura_feature = Tamura_extractor.featureExtract(imgs)
# Daubechies_Wavelets_feature = Daubechies_Wavelets_extractor.featureExtract(imgs)
# print(Daubechies_Wavelets_feature.shape)
# ZernikeMoments_feature = ZernikeMoments_extractor.featureExtract(imgs)
# print(ZernikeMoments_feature.shape)
# Haralick_feature = Haralick_extractor.featureExtract(imgs)
# print(Haralick_feature.shape)
# GIST_feature = GIST_extractor.featureExtract(imgs)
# print(GIST_feature.shape)
# DenseSIFT_feature = DenseSIFT_extractor.featureExtract(imgs)
# print(DenseSIFT_feature.shape)

# DCT_feature = DCT_extractor.featureExtract(imgs)
# print(DCT_feature.shape)



# np.savez("features.npz", cvHOG_feature=cvHOG_feature, Color_feature=Color_feature, raw_feature=raw_feature, EOH_feature=EOH_feature, HuMoments_feature=HuMoments_feature, SIFT_feature=SIFT_feature, LBP_feature=LBP_feature, Gabor_feature=Gabor_feature, Tamura_feature=Tamura_feature)

# append more features on top of the features.npz file
# features = np.load("features.npz")
# cvHOG_feature = features['cvHOG_feature']



**Load Features**

In [12]:
features_file = np.load("features.npz")

imgs_features = np.empty((imgs.shape[0], 0))

features_count = 0

for features in features_file.files[:-1]:
    imgs_features = np.concatenate((imgs_features, features_file[features]), axis=1)
    print(f"Feature {features} appended {features_file[features].shape}")
    features_count += 1

print(f"Total features appended: {features_count}")
# imgs_features = np.concatenate((imgs_features, Daubechies_Wavelets_feature, ZernikeMoments_feature, Haralick_feature, GIST_feature, DenseSIFT_feature), axis=1)



Feature cvHOG_feature appended (110000, 2880)
Feature Color_feature appended (110000, 64)
Feature raw_feature appended (110000, 3072)
Feature EOH_feature appended (110000, 256)
Feature HuMoments_feature appended (110000, 7)
Feature SIFT_feature appended (110000, 128)
Feature LBP_feature appended (110000, 59)
Feature Gabor_feature appended (110000, 24)
Feature Tamura_feature appended (110000, 3)
Feature cooccurrence_feature appended (110000, 4)
Feature Daubechies_Wavelets_feature appended (110000, 4)
Feature ZernikeMoments_feature appended (110000, 25)
Feature Haralick_feature appended (110000, 13)
Feature GIST_feature appended (110000, 13)
Total features appended: 14


In [13]:
# features_file = np.load("features.npz")
# # cvHOG_feature = features_file['cvHOG_feature']
# # Color_feature = features_file['Color_feature']
# # raw_feature = features_file['raw_feature']
# # EOH_feature = features_file['EOH_feature']
# # HuMoments_feature = features_file['HuMoments_feature']
# # SIFT_feature = features_file['SIFT_feature']
# # LBP_feature = features_file['LBP_feature']
# Gabor_feature = features_file['Gabor_feature']
# Tamura_feature = features_file['Tamura_feature']
# cooccurrence_feature = features_file['cooccurrence_feature']
# Daubechies_Wavelets_feature = features_file['Daubechies_Wavelets_feature']
# ZernikeMoments_feature = features_file['ZernikeMoments_feature']
# Haralick_feature = features_file['Haralick_feature']
# GIST_feature = features_file['GIST_feature']
# # DenseSIFT_feature = features_file['DenseSIFT_feature']
# # DCT_feature = features_file['DCT_feature']

# np.savez("sba.npz", Gabor_feature=Gabor_feature, Tamura_feature=Tamura_feature, cooccurrence_feature=cooccurrence_feature, Daubechies_Wavelets_feature=Daubechies_Wavelets_feature, ZernikeMoments_feature=ZernikeMoments_feature, Haralick_feature=Haralick_feature, GIST_feature=GIST_feature)


In [14]:
print(imgs_features.shape)

(110000, 6552)


**Reduce Dimension**

In [15]:
print("begin pca")
pca = PCA(0.88, copy=False)
pca.fit(imgs_features)
imgs_features = pca.fit_transform(imgs_features)
print("pca done")

# skHOG_feature = skHOG_extractor.featureExtract(imgs)
# Color_feature = Color_extractor.featureExtract(imgs)
# print(Color_feature.shape)
# smoothRaw_feature = smoothRaw_extractor.featureExtract(imgs)
# SIFT_feature = SIFT_extractor.featureExtract(imgs)

# print(HuMoments_feature.shape)
# LBP_feature = LBP_extractor.featureExtract(imgs)
# print(LBP_feature.shape)
# cooccurrence_feature = cooccurrence_extractor.featureExtract(imgs)
# print(cooccurrence_feature.shape)
# imgs_features = np.concatenate((imgs_features, EOH_feature, Color_feature, HuMoments_feature), axis=1)
# pca = PCA(n_components=400)
# pca.fit(imgs_features)
# imgs_features = pca.fit_transform(imgs_features)
# print("pca done")
print(imgs_features.shape)

begin pca
pca done
(110000, 472)


In [16]:
# np.savez("pca_features.npz", imgs_features=imgs_features)

**Split**

In [17]:
# train_images = imgs_features[5000:]
# train_labels = labels[5000:]
# test_images = imgs_features[:5000]
# test_labels = labels[:5000]
train_images, test_images, train_labels, test_labels = train_test_split(
    imgs_features[:50000], labels[:50000], test_size=0.1, random_state=42
)

**Train**

In [18]:
model1 = SVC(kernel='rbf', C=5, random_state=42)

In [19]:
# model2 = KNeighborsClassifier(n_neighbors=10, p=1, metric='minkowski', weights='distance')

In [20]:
# model3 = LogisticRegression(max_iter=2000)

In [21]:
# ensemble_model = VotingClassifier(estimators=[('svm', model1), ('knn', model2), ('lr', model3)], voting='soft')

**Load existing features from file**

In [22]:
# load features
features_file = np.load("pca_features.npz")
imgs_features = features_file['imgs_features']

**Train's Test**

In [23]:
train_images = np.concatenate((train_images, imgs_features[50000:100000]), axis=0)
train_labels = np.concatenate((train_labels, labels[50000:100000]), axis=0)
model1.fit(train_images, train_labels)

SVC(C=5, random_state=42)

In [24]:
train_predictions = model1.predict(test_images)
accuracy = accuracy_score(test_labels, train_predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.798


**Test**

In [25]:
predictions = model1.predict(imgs_features[100000:])

# Create a dataframe with the image names and predictions
result_data = pd.DataFrame({'im_name': test_image_names, 'label': predictions})

# Save the result to a CSV file
output_file_path = "predict.csv"
result_data.to_csv(output_file_path, index=False)